In [1]:
import pandas as pd
from sqlalchemy import create_engine
import urllib
from rich.console import Console
import time

# Inicia contador de tiempo
start_time = time.time()
console = Console()

# 🔐 Parámetros de conexión
server = 'KOFMXAZMND01'
database = 'DATAKOF_MX_USERS'

# 🧩 Cadena de conexión para autenticación por Windows
params = urllib.parse.quote_plus(
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

# 🎯 Crear conexión con SQLAlchemy
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# 📥 Consulta SQL
query = """
SELECT DISTINCT
        c.[Num_Cliente] AS ID_SAP,
        c.[NombreCliente] AS NOMBRE,
        c.[Cadena] AS CADENA,
        c.[Coord_X] AS LONGITUD,
        c.[DescGEC] AS GEC_SAP,
        c.[Coord_Y] AS LATITUD,
        c.[ModalidadVentas] AS MODALIDAD,
        c.[Descripción Estado] AS ZONA,
        c.[Zona Ventas   U O ] AS UO,
        c2.[_NIVEL_ESTRATEGICO_1] AS CANAL,
        c2.[_NIVEL_ESTRATEGICO_2] AS GRUPO_RM1,
        c2.[_NIVEL_ESTRATEGICO_3] AS NIVEL_ESTRATEGICO,
        c2.[CANAL_KOF] AS CANAL_KOF_KEY,
        c2.[CANAL_KOF_DESC] AS CANAL_KOF_DESC,
        c2.[GRUPO_CANAL_DESC] AS GRUPO_CANAL_SAP,
        ZPV.[MétodoVenta] AS MTDO_VTA_ZPV,
        ZPV.[Ruta] AS RUTA_ZPV,
        ZPV.[Frecuencia de Visita] AS FV_ZPV,
        ZPV.[Ritmo] AS RITMO_ZPV,
        ZTK.[MétodoVenta] AS MTDO_VTA_ZTK,
        ZTK.[Ruta] AS RUTA_ZTK,
        ZTK.[Frecuencia de Visita] AS FV_ZTK,
        ZTK.[Ritmo] AS RITMO_ZTK,
        ZJV.[MétodoVenta] AS MTDO_VTA_ZJV,
        ZJV.[Ruta] AS RUTA_ZJV,
        ZJV.[Frecuencia de Visita] AS FV_ZJV,
        ZJV.[Ritmo] AS RITMO_ZJV,
        c.[Centro Suministro] AS CENTRO_SUMINISTRO,
        c.[ZonaTransporte] AS RUTA_REPARTO
FROM 
    [DATAKOF_MX_USERS].[kof].[MEX_KOF_SAP_CLIENTE_FULL] c
    LEFT JOIN [DATAKOF_MX_USERS].[kof].[MEX_KOF_SAP_CLIENTE_FULL] ZPV
        ON c.[Num_Cliente] = ZPV.[Num_Cliente]
        AND ZPV.[Tipo Plan de Visita] = 'ZPV'
        AND ZPV.[Estatus del Cliente] NOT IN ('B','L')
    LEFT JOIN [DATAKOF_MX_USERS].[kof].[MEX_KOF_SAP_CLIENTE_FULL] ZJV
        ON c.[Num_Cliente] = ZJV.[Num_Cliente]
        AND ZJV.[Tipo Plan de Visita] = 'ZJV'
        AND ZJV.[Estatus del Cliente] NOT IN ('B','L')
    LEFT JOIN [DATAKOF_MX_USERS].[kof].[MEX_KOF_SAP_CLIENTE_FULL] ZTK
        ON c.[Num_Cliente] = ZTK.[Num_Cliente]
        AND ZTK.[Tipo Plan de Visita] = 'ZTK'
        AND ZTK.[Estatus del Cliente] NOT IN ('B','L')
    LEFT JOIN [kof].[MEX_KOF_SAP_DIM_CLIENTE] c2
        ON c.[Num_Cliente] = c2.[CLIENTE]
WHERE 
    c.[Tipo Plan de Visita] IN ('ZPV', 'ZJV', 'ZTK')
    AND c.[Estatus del Cliente] NOT IN ('B','L')
ORDER BY c.[Num_Cliente]
"""

# 🧪 Ejecutar la consulta
print("⏳ Ejecutando consulta...")
df_clientes = pd.read_sql(query, engine)

# Línea decorativa de cierre
console.rule("[bold magenta]✅ Transformación guardada como 'base_clientes_final.csv'")

# Calcular tiempo de ejecución
elapsed_time = time.time() - start_time
formatted_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

# Mostrar resumen
console.rule("[bold blue]📊 Resumen de la ejecución")
console.print(f"🕒 Tiempo de ejecución total: [bold green]{formatted_time}[/bold green]")
console.print(f"📈 Filas totales: [bold cyan]{len(df_clientes):,}[/bold cyan]")


⏳ Ejecutando consulta...


──────────────────────────── ✅ Transformación guardada como 'base_clientes_final.csv' ────────────────────────────

─────────────────────────────────────────── 📊 Resumen de la ejecución ────────────────────────────────────────────

🕒 Tiempo de ejecución total: 00:18:03

📈 Filas totales: 807,237

🔢 Suma total por medida:

ID_SAP: 478,163,318,004,899.00

In [2]:
print(df_clientes.head())

        ID_SAP               NOMBRE CADENA          LONGITUD GEC_SAP  \
0  100000195.0          ABTS LOLITA   9999  -99.064707000000     ORO   
1  100000204.0     ABTS SUPER MARIO   9999  -99.035621666667     ORO   
2  100000205.0             LOCAL 33   9999  -99.056411666667     ORO   
3  100000206.0  ABTS GANU MEZQUITAL   9999  -99.050032850000     ORO   
4  100000208.0           ABTS OASIS   9999  -99.059125000000  BRONCE   

           LATITUD MODALIDAD          ZONA      UO        CANAL  ...  \
0  19.492514800000        KD  VDM Poniente  MR0025  Tradicional  ...   
1  19.502573333333        KG  VDM Poniente  MR0022  Tradicional  ...   
2  19.491526666667        KD  VDM Poniente  MR0025  Tradicional  ...   
3  19.489654860000        KD  VDM Poniente  MR0025  Tradicional  ...   
4  19.492776666667        KG  VDM Poniente  MR0025  Tradicional  ...   

  MTDO_VTA_ZTK RUTA_ZTK FV_ZTK RITMO_ZTK MTDO_VTA_ZJV RUTA_ZJV FV_ZJV  \
0          1DA   TEWA13    MJS        01          1DA   TEF10

In [3]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807237 entries, 0 to 807236
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ID_SAP             807237 non-null  float64
 1   NOMBRE             807237 non-null  object 
 2   CADENA             807236 non-null  object 
 3   LONGITUD           807237 non-null  object 
 4   GEC_SAP            807237 non-null  object 
 5   LATITUD            807237 non-null  object 
 6   MODALIDAD          807237 non-null  object 
 7   ZONA               807237 non-null  object 
 8   UO                 807237 non-null  object 
 9   CANAL              807237 non-null  object 
 10  GRUPO_RM1          807237 non-null  object 
 11  NIVEL_ESTRATEGICO  807237 non-null  object 
 12  CANAL_KOF_KEY      807237 non-null  object 
 13  CANAL_KOF_DESC     807237 non-null  object 
 14  GRUPO_CANAL_SAP    807237 non-null  object 
 15  MTDO_VTA_ZPV       804257 non-null  object 
 16  RU

In [4]:
# Días de la semana
dias_semana = ['L', 'M', 'R', 'J', 'V', 'S', 'D']

# Columnas FV por TPV
columnas_fv = [col for col in df_clientes.columns if col.startswith('FV')]

# Insertar columnas nuevas justo después de cada columna Fv
for col_fv in columnas_fv:
    tpv = col_fv.split('_')[1]  # Ej. 'ZGC'
    
    # Generar nuevas columnas
    nuevas_columnas = {}
    for dia in dias_semana:
        nueva_col = f'{dia} {tpv}'
        nuevas_columnas[nueva_col] = df_clientes[col_fv].apply(lambda x: 1 if dia in str(x) else 0)
    
    # Encontrar la posición donde está la columna 'Fv <TPV>'
    idx = df_clientes.columns.get_loc(col_fv) + 1
    
    # Insertar las columnas en esa posición
    for i, (col_name, serie) in enumerate(nuevas_columnas.items()):
        df_clientes.insert(loc=idx + i, column=col_name, value=serie)

df_clientes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807237 entries, 0 to 807236
Data columns (total 50 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ID_SAP             807237 non-null  float64
 1   NOMBRE             807237 non-null  object 
 2   CADENA             807236 non-null  object 
 3   LONGITUD           807237 non-null  object 
 4   GEC_SAP            807237 non-null  object 
 5   LATITUD            807237 non-null  object 
 6   MODALIDAD          807237 non-null  object 
 7   ZONA               807237 non-null  object 
 8   UO                 807237 non-null  object 
 9   CANAL              807237 non-null  object 
 10  GRUPO_RM1          807237 non-null  object 
 11  NIVEL_ESTRATEGICO  807237 non-null  object 
 12  CANAL_KOF_KEY      807237 non-null  object 
 13  CANAL_KOF_DESC     807237 non-null  object 
 14  GRUPO_CANAL_SAP    807237 non-null  object 
 15  MTDO_VTA_ZPV       804257 non-null  object 
 16  RU